In [2]:
import pandas as pd

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from lxml import html
from bs4 import BeautifulSoup
import requests
import time

In [35]:
# def scrape_abercrombie(url):
#     try:
#         # Set Chrome options for headless mode
#         chrome_options = Options()
#         chrome_options.add_argument("--headless")
        
#         # Initialize the WebDriver
#         with webdriver.Chrome(options=chrome_options) as driver:
#             driver.get(url)
            
#             # Wait for the page to fully load
#             WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body')))
            
#             # Get the page source after interactions
#             page_source = driver.page_source
        
#         # Parse the page source with Beautiful Soup
#         soup = BeautifulSoup(page_source, "html.parser")
        
#         data_title = soup.find("h1", class_="product-title-component")
#         # Find the <div> tag with class "product__description rte"
#         data_element = soup.find("h4", class_="h4 fabric-care-mfe__label")
        
#         if data_element:
#             return data_title.text, data_element.text.strip()
#         else:
#             return "Link no longer available."
            
#     except Exception as e:
#         return f"An error occurred: {str(e)}"

In [29]:
#WORKS
def process_data(items):
    processed_items = []

    for entry in items:
        # Check if the entry is a tuple and has exactly two elements
        if isinstance(entry, tuple) and len(entry) == 2:
            item, materials = entry
            # Initialize dictionary with the item name
            item_dict = {'item': item}
            
            # Split the materials string by '/'
            components = materials.split('/')
            print(components)
            
            # Process each component
            for component in components:
                # Normalize the component
                component = component.strip()
                # Check if the component starts with "Body:"
                if component.startswith('Body:'):
                    # Try to split by ':'
                    material_name, composition = component.split(':', 1)  # Splits only on the first occurrence
                    material_name = material_name.strip()
                    composition = composition.strip()
                    # Split the composition into segments by space to isolate the percentage and material
                    parts = composition.split()
                    
                    for part in parts:
                        # Look for the percentage part
                        if '%' in part:
                            # Try to split by '%'
                            try:
                                percentage = int(part.split('%')[0])
                                # The material name usually follows the percentage
                                material_index = parts.index(part) + 1
                                if material_index < len(parts):
                                    material = parts[material_index].strip()
                                    # Remove any commas or non-alphanumeric characters from material name
                                    material = ''.join(char for char in material if char.isalnum() or char.isspace())
                                    # Map to the dictionary, converting material name to lowercase
                                    item_dict[material.lower()] = percentage
                            except ValueError:
                                continue  # If conversion fails, skip this part

            # Append the processed dictionary to the result list
            processed_items.append(item_dict)
        else:
            # Handle entries that are not correctly formatted
            processed_items.append({"error": "error"})

    return processed_items

# Example usage with corrected structure:
items = [('Cropped Squareneck Rib Tank', 'Body:95% Cotton, 5% Elastane'),
 ('Curve Love High Rise Dad Short',
  'Pocket Bag:70% Polyester, 30% Cotton / Body:99% Cotton, 1% Elastane'),
 ('Vintage Sunday Short',
  'Body:70% Cotton, 30% Polyester / Pocket Bag:60% Cotton, 40% Polyester'),
 ('Summer Destination Vintage Sunday Crew',
  'Embroidery:100% Polyester / Body:70% Cotton, 30% Polyester'),
 ('Linen-Blend Pull-On Pant',
  'Lining:100% Viscose / Body:55% Linen, 45% Viscose'),
 ('Linen-Blend Set Top',
  'Lining:100% Viscose / Elastic:70% Polyester, 30% Elastane / Body:55% Linen, 45% Viscose'),
 ('The A&F Scarlett Linen-Blend Wrap Mini Skort',
  'Lining:100% Viscose / Body:55% Linen, 45% Viscose'),
 ('Premium Linen Scallop-Hem Set Top', 'Lining:100% Cotton / Body:100% Linen'),
 ('Premium Linen Scallop-Hem Mini Skort',
  'Lining:100% Cotton / Body:100% Linen'),
 ('Clean Waist Premium Linen Trouser',
  'Pocket Bag:100% Cotton / Body:100% Linen'),
 ('Essential Scoopneck Tank', 'Body:95% Cotton, 5% Elastane'),
 ('Curve Love High Rise 4" Mom Short',
  'Body:99% Cotton, 1% Elastane / Pocket Bag:50% Cotton, 50% Polyester'),
 'Link no longer available.',
 'Link no longer available.',
 'Link no longer available.',
 'Link no longer available.',
 'Link no longer available.',
 'Link no longer available.',
 'Link no longer available.']

processed_items = process_data(items)
for item in processed_items:
    print(item)


['Body:95% Cotton, 5% Elastane']
['Pocket Bag:70% Polyester, 30% Cotton ', ' Body:99% Cotton, 1% Elastane']
['Body:70% Cotton, 30% Polyester ', ' Pocket Bag:60% Cotton, 40% Polyester']
['Embroidery:100% Polyester ', ' Body:70% Cotton, 30% Polyester']
['Lining:100% Viscose ', ' Body:55% Linen, 45% Viscose']
['Lining:100% Viscose ', ' Elastic:70% Polyester, 30% Elastane ', ' Body:55% Linen, 45% Viscose']
['Lining:100% Viscose ', ' Body:55% Linen, 45% Viscose']
['Lining:100% Cotton ', ' Body:100% Linen']
['Lining:100% Cotton ', ' Body:100% Linen']
['Pocket Bag:100% Cotton ', ' Body:100% Linen']
['Body:95% Cotton, 5% Elastane']
['Body:99% Cotton, 1% Elastane ', ' Pocket Bag:50% Cotton, 50% Polyester']
{'item': 'Cropped Squareneck Rib Tank', 'cotton': 95, 'elastane': 5}
{'item': 'Curve Love High Rise Dad Short', 'cotton': 99, 'elastane': 1}
{'item': 'Vintage Sunday Short', 'cotton': 70, 'polyester': 30}
{'item': 'Summer Destination Vintage Sunday Crew', 'cotton': 70, 'polyester': 30}
{'item

In [36]:
df = pd.read_csv("../../data/youtube_data/abercrombie_and_fitch_youtube_data.csv")


(14, 3)

In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def scrape_abercrombie(url):
    try:
        # Set Chrome options for headless mode
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        
        # Initialize the WebDriver
        with webdriver.Chrome(options=chrome_options) as driver:
            driver.get(url)
            
            # Wait for the page to fully load
            WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body')))
            
            # Get the page source after interactions
            page_source = driver.page_source
        
        # Parse the page source with Beautiful Soup
        soup = BeautifulSoup(page_source, "html.parser")
        
        data_title = soup.find("h1", class_="product-title-component")
        data_element = soup.find("h4", class_="h4 fabric-care-mfe__label")
        
        if data_title and data_element:
            item = data_title.text.strip()
            materials = data_element.text.strip()
            return process_item_data(item, materials)
        else:
            return {"error": "Link no longer available or missing data."}
            
    except Exception as e:
        return {"error": f"An error occurred: {str(e)}"}

def process_item_data(item, materials):
    item_dict = {'item': item}
    
    # Process materials
    if 'Body:' in materials:
        components = materials.split('/')
        for component in components:
            component = component.strip()
            if component.startswith('Body:'):
                _, composition = component.split(':', 1)
                parts = composition.strip().split()
                
                for part in parts:
                    if '%' in part:
                        try:
                            percentage = int(part.split('%')[0])
                            material_index = parts.index(part) + 1
                            if material_index < len(parts):
                                material = parts[material_index].strip()
                                material = ''.join(char for char in material if char.isalnum() or char.isspace())
                                item_dict[material.lower()] = percentage
                        except ValueError:
                            continue
    else:
        item_dict['materials'] = "No body material info available."
        
    return item_dict

# # Example usage:
# urls = ['https://www.abercrombie.com/shop/us/p/high-rise-90s-cutoff-short-55097319?categoryId=12203&faceout=model&seq=03',
#         'https://www.abercrombie.com/shop/us/p/crochet-style-v-neck-mini-dress-53805823?categoryId=12265&faceout=life&seq=02',
#         'https://www.abercrombie.com/shop/us/p/oversized-gauzy-shirt-53457331?categoryId=6570723&faceout=model&seq=01',
#         'https://www.abercrombie.com/shop/us/p/crochet-style-maxi-dress-53806825?categoryId=12265&faceout=life&seq=02']



# for url in urls:
#     item_info = scrape_abercrombie(url)
#     print(item_info)   



{'item': 'High Rise 90s Cutoff Short', 'cotton': 99, 'elastane': 1}
{'item': 'Crochet-Style V-Neck Mini Dress', 'cotton': 100}
{'item': 'Oversized Gauzy Shirt', 'cotton': 100}
{'item': 'Crochet-Style Maxi Dress', 'cotton': 100}
